# Daten zu Feiertagen

### Feiertage von 2019 bis 2023 über API abrufen
- über API: https://date.nager.at/
- API URL: https://date.nager.at/api/v3/PublicHolidays/{year}/DE
- Filterung für Berlin (DE-BE)

In [7]:
import requests
import pandas as pd

# für Berlin
def get_feiertage(year):
    # URL für die Feiertage eines bestimmten Jahres in Deutschland
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/DE"

    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        berlin_feiertage = [
            {"Datum": entry["date"], "Feiertag": entry["localName"]} 
            for entry in data 
            if entry["counties"] is None or "DE-BE" in entry["counties"]
        ]
        return berlin_feiertage
    else:
        print(f"Fehler beim Abrufen der Feiertage für {year}")
        return []

# alle von 2019 bis 2023 
feiertage_list = []
for year in range(2019, 2024):
    feiertage_list.extend(get_feiertage(year))

# in df umwandeln
df_feiertage = pd.DataFrame(feiertage_list)

# Datum in Format datetime umwandeln / ISO-Format
df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"])

# als csv speichern
# df_feiertage.to_csv("feiertage_berlin_2019_2023.csv", index=False)

### Datensatz Überblick

In [59]:
import pandas as pd

df_feiertage = pd.read_csv("daten/feiertage_berlin_2019_2023.csv")

In [60]:
df_feiertage.head(3)

,Datum,Feiertag
0,2019-01-01,Neujahr
1,2019-03-08,Internationaler Frauentag
2,2019-04-19,Karfreitag


In [43]:
df_feiertage.tail(3)

,Datum,Feiertag
48,2023-10-03,Tag der Deutschen Einheit
49,2023-12-25,Erster Weihnachtstag
50,2023-12-26,Zweiter Weihnachtstag


In [65]:
# Überblick
df_feiertage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datum     51 non-null     datetime64[ns]
 1   Feiertag  51 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 948.0+ bytes


In [45]:
# Statistische Zusammenfassung (für numerische Spalten)
df_feiertage.describe()

,Datum,Feiertag
count,51,51
unique,51,11
top,2019-01-01,Neujahr
freq,1,5


### Fehlende Werte

In [61]:
df_feiertage.isnull().sum()

Datum       0
Feiertag    0
dtype: int64

### Liste mit allen Tagen erstellen & Feiertage markieren
- Feiertag = 1
- kein Feiertag = 0

In [62]:
# 1. alle Tage zwischen 2019-01-01 und 2023-12-31
all_dates = pd.date_range(start="2019-01-01", end="2023-12-31", freq="D")

# 2. df 
df_all_days = pd.DataFrame({"Datum": all_dates})

# 3. neue Spalte mit 'Feiertag' mit Wert 0 (default) 
df_all_days["Feiertag"] = 0

# 4. Datumsformat
df_all_days["Datum"] = pd.to_datetime(df_all_days["Datum"], format="%Y-%m-%d")
df_feiertage["Datum"] = pd.to_datetime(df_feiertage["Datum"], format="%Y-%m-%d")

# 5. Feiertage merhalten Wert 1
df_all_days.loc[df_all_days["Datum"].isin(df_feiertage["Datum"]), "Feiertag"] = 1

In [63]:
df_all_days.head(3)

,Datum,Feiertag
0,2019-01-01,1
1,2019-01-02,0
2,2019-01-03,0


In [64]:
(df_all_days.tail(10))

,Datum,Feiertag
1816,2023-12-22,0
1817,2023-12-23,0
1818,2023-12-24,0
1819,2023-12-25,1
1820,2023-12-26,1
1821,2023-12-27,0
1822,2023-12-28,0
1823,2023-12-29,0
1824,2023-12-30,0
1825,2023-12-31,0


In [58]:
# als csv speichern
# df_all_days.to_csv("data/feiertage_komplett.csv", index=False, encoding="utf-8")